In [1]:
%pip install -U pip
%pip install -q accelerate -U
%pip install -q ipywidgets
%pip install -q transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import hashlib
import math

import polars as pl


import polars as pl
import numpy as np


from torch import nn, Tensor
from tqdm import trange, tqdm

import hephaestus as hp
import torch
from torch.utils.tensorboard import SummaryWriter

In [3]:
df = pl.read_csv("../data/diamonds.csv")
df.head()

carat,cut,color,clarity,depth,table,price,x,y,z
f64,str,str,str,f64,f64,i64,f64,f64,f64
0.23,"""Ideal""","""E""","""SI2""",61.5,55.0,326,3.95,3.98,2.43
0.21,"""Premium""","""E""","""SI1""",59.8,61.0,326,3.89,3.84,2.31
0.23,"""Good""","""E""","""VS1""",56.9,65.0,327,4.05,4.07,2.31
0.29,"""Premium""","""I""","""VS2""",62.4,58.0,334,4.2,4.23,2.63
0.31,"""Good""","""J""","""SI2""",63.3,58.0,335,4.34,4.35,2.75


In [4]:
df = hp.scale_numeric(df)
df.head()

carat,cut,color,clarity,depth,table,price,x,y,z
f64,str,str,str,f64,f64,f64,f64,f64,f64
-1.198157,"""Ideal""","""E""","""SI2""",-0.17409,-1.099662,-0.904087,-1.587823,-1.536181,-1.571115
-1.24035,"""Premium""","""E""","""SI1""",-1.360726,1.585514,-0.904087,-1.64131,-1.658759,-1.741159
-1.198157,"""Good""","""E""","""VS1""",-3.384987,3.375631,-0.903836,-1.498677,-1.457382,-1.741159
-1.071577,"""Premium""","""I""","""VS2""",0.454129,0.242926,-0.902081,-1.364959,-1.317293,-1.287708
-1.029384,"""Good""","""J""","""SI2""",1.082348,0.242926,-0.901831,-1.240155,-1.212227,-1.117663


In [5]:
train_fraction = 0.8
n_train = int(train_fraction * len(df))
train_test_df = df.select(pl.all().exclude(["price", "hash"]))

train_test_df = train_test_df.sample(fraction=1.0)

train_df, test_df = train_test_df.head(n_train), train_test_df.tail(
    len(train_test_df) - n_train
)

In [6]:
train_df.head(), train_df.shape

(shape: (5, 9)
 ┌───────────┬─────────┬───────┬─────────┬───┬───────────┬───────────┬───────────┬───────────┐
 │ carat     ┆ cut     ┆ color ┆ clarity ┆ … ┆ table     ┆ x         ┆ y         ┆ z         │
 │ ---       ┆ ---     ┆ ---   ┆ ---     ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
 │ f64       ┆ str     ┆ str   ┆ str     ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
 ╞═══════════╪═════════╪═══════╪═════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
 │ -1.198157 ┆ Ideal   ┆ E     ┆ SI2     ┆ … ┆ -1.099662 ┆ -1.587823 ┆ -1.536181 ┆ -1.571115 │
 │ -1.24035  ┆ Premium ┆ E     ┆ SI1     ┆ … ┆ 1.585514  ┆ -1.64131  ┆ -1.658759 ┆ -1.741159 │
 │ -1.198157 ┆ Good    ┆ E     ┆ VS1     ┆ … ┆ 3.375631  ┆ -1.498677 ┆ -1.457382 ┆ -1.741159 │
 │ -1.071577 ┆ Premium ┆ I     ┆ VS2     ┆ … ┆ 0.242926  ┆ -1.364959 ┆ -1.317293 ┆ -1.287708 │
 │ -1.029384 ┆ Good    ┆ J     ┆ SI2     ┆ … ┆ 0.242926  ┆ -1.240155 ┆ -1.212227 ┆ -1.117663 │
 └───────────┴─────────┴───────┴───

In [7]:
special_tokens = np.array(
    [
        "missing",
        "[MASK]",
        "[NUMERIC]",
        "<unk>",
        ":",
        ",",
        "<row-start>",
        "<row-end>",
    ]
)

tokens = np.array(
    [
        "missing",
        "[MASK]",
        "[NUMERIC]",
        "<unk>",
        ":",
        ",",
        "<row-start>",
        "<row-end>",
    ]
)

ds = hp.TabularDataset(
    train_df,
    tokens,
    special_tokens=special_tokens,
    shuffle_cols=True,
    max_row_length=50,
)
tokens = np.unique(
    np.concatenate(
        (
            special_tokens,
            np.array(
                [
                    "<numeric>",
                ]
            ),
        )
    )
)
tokens
print(len(ds[0]))

38


In [8]:
if torch.backends.mps.is_built():
    device_name = "mps"
elif torch.cuda.is_available():
    device_name = "cuda"
else:
    device_name = "cpu"
device = torch.device(device_name)
print(device)

cuda


In [9]:
# model = hp.TransformerModel(device=device).to(device)

In [10]:
# model.load_state_dict(torch.load("models/2023-08-24T17:54:26_non-scale.pt"))

In [11]:
# model.eval()
# with torch.no_grad():
#     class_pred, numeric_pred = model(ds[0])

In [12]:
# class_pred.shape, numeric_pred.shape

In [13]:
# hp.evaluate_bert(model, ds, 0)

In [67]:
class PricePredictor(nn.Module):
    def __init__(self):
        super(PricePredictor, self).__init__()
    
        self.transformer_model = hp.TransformerModel(device=device).to(device)
        self.transformer_model.load_state_dict(torch.load("models/2023-08-24T17:54:26_non-scale.pt"))
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear((self.transformer_model.embedding_dim+1)*41, 500),
            nn.ReLU(), 
            nn.Linear(500, 41),
            nn.ReLU(), 
            nn.Linear(41, 1), 
            nn.ReLU(),# nn.Linear(1,1),
        ).to(device)
        print("Done Init")
    def forward(self, input_):
        print("In Forward")
        class_output, numeric_output = self.transformer_model(input_, finetune=True)
        class_output = class_output.squeeze()
        numeric_output = numeric_output.reshape(41, 1)
        # return class_output, numeric_output
        output = torch.cat((class_output, numeric_output), dim=1)
        output = output.unsqueeze(0)
        res = self.fc(output)
        return res
    
pp = PricePredictor()
res = pp(ds[0])
                
        
        

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 30527. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Done Init
In Forward
Fine Tune Enabled


In [68]:
res.shape

torch.Size([1, 1])

In [69]:
res

tensor([[0.0568]], device='cuda:0', grad_fn=<ReluBackward0>)

In [40]:
pp.transformer_model.embedding_dim

768

In [38]:
n.reshape(41, 1).shape

torch.Size([41, 1])

In [51]:
nccat = torch.cat((c, n.reshape(41, 1)), dim=1)
nccat.shape

torch.Size([41, 769])

In [64]:
fc = nn.Sequential(
    # Flatten the tensor
    nn.Flatten(),
    
    # First fully connected layer
    nn.Linear(41*769, 1000),  # You can choose any number for the hidden layer size.
    nn.ReLU(),  # Activation function
    
    # Additional fully connected layers (you can add more if necessary)
    nn.Linear(1000, 100),
    nn.ReLU(),
    
    # Final layer to get a single number
    nn.Linear(100, 1)
).to(device)
print(nccat.shape)
fc(nccat.unsqueeze(0))


torch.Size([41, 769])


tensor([[0.0315]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [62]:
nccat.unsqueeze(0)

tensor([[[-0.7662, -0.1383, -0.1950,  ...,  0.7451,  0.0498, -0.6614],
         [-0.7271,  0.5371,  0.0497,  ...,  0.2285, -0.5835, -1.1123],
         [-0.0311,  0.2797,  0.0780,  ...,  0.4470, -0.4852, -1.1554],
         ...,
         [-0.6136,  0.6360, -0.0600,  ...,  0.2460, -0.6227, -0.8628],
         [-0.6318,  0.4673,  0.6333,  ...,  0.1850,  0.2283, -0.9279],
         [-0.7704,  0.2625,  0.0889,  ...,  0.1702,  0.0282, -0.9946]]],
       device='cuda:0', grad_fn=<UnsqueezeBackward0>)

In [65]:
nccat.ndim

2